In [1]:
import pandas as pd
import numpy as np
import math
from functools import partial, reduce
import os
import re

In [2]:
def split_participant_id(df):
    split_up = df['participant_id'].split('-')
    return split_up[2]

In [3]:
input_path = 'C:/Users/lzoeckler/Desktop/gold_standard_classes/4plex_WB_NIH_Pf_TBV'

In [4]:
base = pd.read_csv('{}/formatted_4plex_WB_NIH_Pf_TBV.csv'.format(input_path))
base['Study ID number'] = base.apply(split_participant_id, axis=1)
base['Study ID number'] = base['Study ID number'].apply(int)
base['unformatted_days'] = base['unformatted_days'].apply(int)
base.head()

,study_id,sample_id,participant_id,sample_type,country,lat,long,age_yrs,date,unformatted_days,...,quansys_LDH_Pv_pg_ml,quansys_LDH_Pf_pg_ml,quansys_CRP_ng_ml,quansys_HRP2_result,quansys_LDH_Pan_result,quansys_LDH_Pv_result,quansys_LDH_Pf_result,quansys_CRP_result,quansys_type,Study ID number
0,12,1 MV-15-301-F D547 (),MV-15-301-F,WB,NaN,NaN,NaN,NaN,2016-09-22,547,...,6.4182,NaN,3911.39702,0,0,0,NaN,NaN,4plex,301
1,12,24 MV-15-301-F D554 (),MV-15-301-F,WB,NaN,NaN,NaN,NaN,2016-09-30,554,...,8.7457,NaN,1383.32281,0,0,0,NaN,NaN,4plex,301
2,12,14 301-F D568 (),MV-15-301-F,WB,NaN,NaN,NaN,NaN,2016-10-13,568,...,5.5062,NaN,195.1877,1,1,0,NaN,NaN,4plex,301
3,12,32 301-F D582 (),MV-15-301-F,WB,NaN,NaN,NaN,NaN,2016-10-27,582,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4plex,301
4,12,2 MV-15-302-k D547 (),MV-15-302-k,WB,NaN,NaN,NaN,NaN,2016-09-22,547,...,6.5968,NaN,1572.37057,0,0,0,NaN,NaN,4plex,302


In [5]:
meta = pd.read_csv('{}/TBV_metadata.csv'.format(input_path))
meta['Visit Date'] = meta['Drug1 Start Date']
meta.rename(columns={'Drug1 Start Date': 'Drug1 started'}, inplace=True)
meta['Drug1 started'] = 1
meta.head()

,Study ID number,Drug1 Name,Drug1 Indication,Drug1 started,Drug1 Stop Date,Visit Date
0,329,ARTEMETHER LUMEFANTRINE,ACCES PALUSTRE,1,2016-07-19,2016-07-17
1,318,COARTEM,ACCES PALUSTRE,1,2016-07-23,2016-07-21
2,487,ARTEMETHER LUMEFANTRINE,ACCES PALUSTRE,1,2016-08-05,2016-08-02
3,457,ARTEMETHER LUMEFANTRINE,ACCES PALUSTRE,1,2016-08-09,2016-08-06
4,412,ARTEMETHER LUMEFANTRINE,ACCES PALUSTRE,1,2016-08-10,2016-08-08


In [6]:
dem = pd.read_csv('{}/dem_data.csv'.format(input_path))
dem.head()

,Study ID number,RDT if done,P. falciparum,P. malariae,P. ovale,Gametocyte falci 1,Gametocyte falci 2,Avg gametocyte,Visit Date,1. Date of Birth,Age at visit,studyday,visit type
0,301,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2016-08-16,1984-06-16,32.19,510.0,scheduled
1,301,Blank,NaN,NaN,NaN,NaN,NaN,0.0,2016-09-05,1984-06-16,32.24,NaN,UNSCH
2,301,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2016-09-15,1984-06-16,32.27,540.0,scheduled
3,301,Negative,NaN,NaN,NaN,NaN,NaN,0.0,2016-09-20,1984-06-16,32.28,NaN,UNSCH
4,301,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2016-09-22,1984-06-16,32.29,547.0,scheduled


In [7]:
combo_dem = dem.merge(meta, how='left', on=['Study ID number', 'Visit Date'])
combo_dem.rename(columns={'Visit Date': 'date'}, inplace=True)
combo_dem.head()

,Study ID number,RDT if done,P. falciparum,P. malariae,P. ovale,Gametocyte falci 1,Gametocyte falci 2,Avg gametocyte,date,1. Date of Birth,Age at visit,studyday,visit type,Drug1 Name,Drug1 Indication,Drug1 started,Drug1 Stop Date
0,301,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2016-08-16,1984-06-16,32.19,510.0,scheduled,NaN,NaN,NaN,NaN
1,301,Blank,NaN,NaN,NaN,NaN,NaN,0.0,2016-09-05,1984-06-16,32.24,NaN,UNSCH,NaN,NaN,NaN,NaN
2,301,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2016-09-15,1984-06-16,32.27,540.0,scheduled,NaN,NaN,NaN,NaN
3,301,Negative,NaN,NaN,NaN,NaN,NaN,0.0,2016-09-20,1984-06-16,32.28,NaN,UNSCH,NaN,NaN,NaN,NaN
4,301,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2016-09-22,1984-06-16,32.29,547.0,scheduled,NaN,NaN,NaN,NaN


In [9]:
combo_all = base.merge(combo_dem, how='left', on=['Study ID number', 'date'])
combo_all.head()

,study_id,sample_id,participant_id,sample_type,country,lat,long,age_yrs,date,unformatted_days,...,Gametocyte falci 2,Avg gametocyte,1. Date of Birth,Age at visit,studyday,visit type,Drug1 Name,Drug1 Indication,Drug1 started,Drug1 Stop Date
0,12,1 MV-15-301-F D547 (),MV-15-301-F,WB,NaN,NaN,NaN,NaN,2016-09-22,547,...,NaN,0.0,1984-06-16,32.29,547.0,scheduled,NaN,NaN,NaN,NaN
1,12,24 MV-15-301-F D554 (),MV-15-301-F,WB,NaN,NaN,NaN,NaN,2016-09-30,554,...,NaN,0.0,1984-06-16,32.31,554.0,scheduled,NaN,NaN,NaN,NaN
2,12,14 301-F D568 (),MV-15-301-F,WB,NaN,NaN,NaN,NaN,2016-10-13,568,...,NaN,0.0,1984-06-16,32.35,568.0,scheduled,NaN,NaN,NaN,NaN
3,12,32 301-F D582 (),MV-15-301-F,WB,NaN,NaN,NaN,NaN,2016-10-27,582,...,NaN,0.0,1984-06-16,32.39,582.0,scheduled,NaN,NaN,NaN,NaN
4,12,2 MV-15-302-k D547 (),MV-15-302-k,WB,NaN,NaN,NaN,NaN,2016-09-22,547,...,NaN,0.0,1973-06-15,43.30,547.0,scheduled,NaN,NaN,NaN,NaN


In [10]:
combo_all.columns

Index(['study_id', 'sample_id', 'participant_id', 'sample_type', 'country',
       'lat', 'long', 'age_yrs', 'date', 'unformatted_days', 'timepoint_days',
       'febrile', 'survey', 'infection_category', 'PCR_pos', 'microscopy_pos',
       'p_spp', 'pf', 'pv', 'pm', 'pk', 'po', 'coinfection', 'HRP2_deletion',
       'quansys_HRP2_pg_ml', 'quansys_LDH_Pan_pg_ml', 'quansys_LDH_Pv_pg_ml',
       'quansys_LDH_Pf_pg_ml', 'quansys_CRP_ng_ml', 'quansys_HRP2_result',
       'quansys_LDH_Pan_result', 'quansys_LDH_Pv_result',
       'quansys_LDH_Pf_result', 'quansys_CRP_result', 'quansys_type',
       'Study ID number', 'RDT if done', 'P. falciparum', 'P. malariae',
       'P. ovale', 'Gametocyte falci 1', 'Gametocyte falci 2',
       'Avg gametocyte', '1. Date of Birth', 'Age at visit', 'studyday',
       'visit type', 'Drug1 Name', 'Drug1 Indication', 'Drug1 started',
       'Drug1 Stop Date'],
      dtype='object')

In [14]:
combo_all.to_csv('{}/combined_all.csv'.format(input_path), index=False)